In [13]:
import pandas as pd

In [14]:
skincare=pd.read_csv('SkinCareDetail.csv')
makeup=pd.read_csv('MakeupDetail.csv')
skincare.shape,makeup.shape

((4397, 11), (4896, 11))

In [15]:
#Combine makeup and skincare data
skincare.insert(8,'Main_Category',["Skincare"]*4397,True)
makeup.insert(8,'Main_Category',["Makeup"]*4896,True)
ulta=pd.concat([skincare,makeup],axis=0)
ulta.index=range(0,len(ulta))
ulta['Price']=[x.split("Original Price")[0] for x in ulta['Price']]
ulta.sample(3)

,Brand,Name,Rating,Price,Link,ID,Reviews,Ingredient,Main_Category,Category,Details,Image
2774,Origins,"Hello, Calm Face Mask with Cannabis Sativa See...",4.6,$32.00,https://www.ulta.com/p/hello-calm-face-mask-wi...,2542047,376.0,"""Water\\aqua\\eau, butylene glycol, prunus arm...",Skincare,Treatment & Serums,"Details\nBenefits\nThis lush, nourishing mask ...",Image is not available
3343,COOLA,Peach Blossom Classic Body Sunscreen SPF 70,3.6,$28.00,https://www.ulta.com/p/peach-blossom-classic-b...,2575596,11.0,"Active: Avobenzone 3.0%, Homosalate 5.0%, Octi...",Skincare,Suncare,Details\nBenefits\n70%+ certified organic ingr...,Image is not available
8371,Morphe,Morphe 2 X Maddie Ziegler Changement Artistry ...,2.9,$18.00,https://www.ulta.com/p/morphe-2-x-maddie-ziegl...,2601774,8.0,Ingredients\nPlease see product packaging for ...,Makeup,Eyes,Details\nShades\nTop Row: Reflex (shimmer whit...,https://media.ulta.com/i/ulta/3974?w=582&h=582...


In [16]:
indexAge=ulta[ ulta['Category'].isin(['Makeup Brushes & Tools' ,'Makeup Bags & Organizers','Skincare Tools',
                                     'Bath & Body Accessories'
                                     ])].index
ulta.drop(indexAge,inplace=True)

In [17]:
def remove_prefix(text, prefix):
    return text[text.startswith(prefix) and len(prefix):].strip()

ulta['Ingredient']=ulta.apply(lambda x: remove_prefix(x['Ingredient'],'Ingredients\n'),axis=1)
ulta['Ingredient']=ulta.apply(lambda x: remove_prefix(x['Ingredient'],'Active:'),axis=1)
ulta['Ingredient']=ulta.apply(lambda x: x['Ingredient'].lower(),axis=1)

In [18]:
import re
ig=ulta['Ingredient'][1951]
print(ig)
def get_ig_list(ig):
    l=re.split('[,]',ig.split(".")[0])
    l=[x.strip().lower().rstrip(".*") for x in l  ]
    l=[re.sub("\d*%","",x).strip() for x in l if not x.startswith(('*',"please","water","aqua","eau","ingredient")) 
       and not x.isdigit() and "may contain" not in x ]
    return sorted(l)

get_ig_list(ig)

avobenzone 3%, homosalate 15%, octisalate 5%, octocrylene 7%.


['avobenzone', 'homosalate', 'octisalate', 'octocrylene']

In [19]:
#Drop lines with <=3 ingredients provided
ulta['Ig_list']=ulta.apply(lambda x: get_ig_list(x['Ingredient']),axis=1)
ulta['len']=ulta.apply(lambda x:len(x['Ig_list']),axis=1)
#ulta['Ig_list'][1951]
indexAge=ulta[ ulta['len']<=3].index
ulta.drop(indexAge,inplace=True)
#ulta = ulta.drop(columns=['Ig_list', 'len'])

In [20]:
#Drop duplicates
ulta.drop_duplicates(subset=["Name",'ID'],inplace=True)
ulta.to_csv('Clean_data',index=None)


In [21]:
from platform import python_version

print(python_version())

3.9.13
